In [1]:
import pandas as pd
import numpy as np 
import os 
import sys
from itertools import chain

In [142]:
sys.path.append(os.path.abspath("Code"))

In [143]:
os.chdir("/home/llan/Desktop/WUR/thesis2")
LABEL_DIR = "shared_data/binary_labels/"


In [1]:
6/400

0.015

In [2]:
20_000 * 0.015

300.0

In [3]:
300/24

12.5

In [201]:
label_data = LABEL_DIR + "Labels.tsv"
tf_list = LABEL_DIR + "TF_list.tsv"
tg_list = LABEL_DIR + "TG_list.tsv"

label_data = pd.read_table(label_data, header=0, index_col=0)
tf_list = pd.read_table(tf_list, header=0).idx.tolist()
tg_list = pd.read_table(tg_list, header=0).idx.tolist()


In [10]:
from DataProcessor import DatasetProcessor

In [4]:
import pickle 
with open("../media2/binary/optimization/tried_configs.txt", "rb") as f:
    a = pickle.load(f)

### creata multilabel file 

In [11]:
os.getcwd()

'/home/llan/Desktop/WUR/thesis2'

In [13]:
ath = pd.read_table("LABELS/Regulations_in_ATRM.tsv", usecols=["TF index", "Target index", "Activate/Repress"])

In [21]:
ath = ath[["TF index", "Target index", "Activate/Repress"]]
ath.columns = ["TF ID", "Target ID", "Activate/Repress"]

In [26]:
ath = ath.loc[ath["Activate/Repress"] != "D"]

In [27]:
def regtoi(x):
    conversion = {"A": 2, "R": 1}
    x = conversion[x]
    return x

ath["Label"] = ath["Activate/Repress"].apply(regtoi)


/tmp/ipykernel_270252/1163911823.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ath["Label"] = ath["Activate/Repress"].apply(regtoi)


In [24]:
ath.to_csv("shared_data/multiclass_labels/Labels.tsv", sep="\t")

In [12]:
ath = pd.read_table("shared_data/multiclass_labels/Labels.tsv", index_col=0)

In [21]:
intersect = ath["TF ID"].isin(label_data["TF ID"]) & ath["Target ID"].isin(label_data["Target ID"])
ath = ath[intersect]

## create data 

In [202]:
bin_split = DatasetProcessor()
bin_split.split_binary_dataset(label_data, tg_list)
bin_split.to_tsv("shared_data/binary_labels/PN_1_1")

In [ ]:
bin_alt_split = DatasetProcessor()
bin_alt_split.split_binary_no_overlap(label_data, tg_list)
bin_alt_split.to_tsv("shared_data/binary_labels/TF_split")

In [25]:
mlc_labels = pd.read_table("shared_data/multiclass_labels/Labels.tsv", header=0, usecols=["TF ID", "Target ID", "Label"])

In [27]:
mlc_split = DatasetProcessor()
mlc_split.split_multiclass_dataset(mlc_labels, tg_list)
mlc_split.to_tsv("shared_data/multiclass_labels/PN_1_1")

In [28]:
mlc_alt_split = DatasetProcessor()
mlc_alt_split.split_multiclass_no_overlap(mlc_labels, tg_list)
mlc_alt_split.to_tsv("shared_data/multiclass_labels/TF_split")

## GO 

In [145]:
label_data

,TF ID,Target ID
0,5,456
1,5,3527
2,5,4626
3,5,7756
4,5,12597
...,...,...
1417,36454,35760
1418,36459,7756
1420,36747,15381
1421,36966,8342


#### label data without IEA entries

In [146]:
wo_iea = pd.read_table("GO/go_wo_iea_n.txt")
wo_comp = pd.read_table("GO/go_wo_comp_n.txt")

In [95]:
go_data = wo_iea.iloc[:, 0].apply(lambda x: list(
            map(int, str(x).split()))).to_dict()

In [96]:
from itertools import chain

values = set(list(chain.from_iterable(list(go_data.values()))))
to = range(len(values))

conversion = {key:val for key, val in zip(values, to)}
func = lambda x: conversion[x]
go_converted = {}
for key, val in go_data.items():
    go_converted[key] = list(map(func, list(val)))

In [97]:
max(list(chain.from_iterable(list(go_converted.values()))))

5488

In [98]:
with open("GO/go_wo_comp_n.txt", "w") as f:
    for key, val in go_converted.items():
        val = " ".join(list(map(str, val)))
        line = f"{key}\t{val}\n"
        f.write(line)

In [56]:
to

range(0, 6809)

In [148]:
wo_comp

,1,4 7 10 11
0,2,12 13 14 15 16 2 0
1,3,17 20 21 22 25 12 27 28 29 30 31 32 33 14 2 34 36
2,4,26 38 39 40 42 43 44 2 46
3,5,47 0 48 1 49 50 51 3 52
4,6,53 54 55
...,...,...
25379,37325,10
25380,37326,1136 10 3431
25381,37327,3431 1077 10
25382,37328,10 87


In [149]:
iea_mask = (label_data["TF ID"].isin(wo_iea.iloc[:,0]) & label_data["Target ID"].isin(wo_iea.iloc[:, 0]))

In [150]:
iea_mask.value_counts()

True    1229
Name: count, dtype: int64

In [151]:
bin_comp_split = DatasetProcessor()
bin_comp_split.split_binary_dataset(label_data, tg_list)
bin_comp_split.to_tsv("shared_data/GO/PN_1_1")

#### label data without computational entries

In [153]:
comp_mask = (label_data["TF ID"].isin(wo_comp.iloc[:, 0]) & label_data["Target ID"].isin(wo_comp.iloc[:,0]))

In [154]:
label_data = label_data[comp_mask]

In [155]:
label_data

,TF ID,Target ID
0,5,456
1,5,3527
2,5,4626
3,5,7756
4,5,12597
...,...,...
1417,36454,35760
1418,36459,7756
1420,36747,15381
1421,36966,8342


In [156]:
tg_list = pd.Series(tg_list)[pd.Series(tg_list).isin(wo_comp.iloc[:,0])].tolist()

In [161]:
bin_comp_split = DatasetProcessor()
bin_comp_split.split_binary_dataset(label_data, tg_list)
bin_comp_split.to_tsv("shared_data/GO/without_comp_go_entries")

In [159]:
20 in label_data["TF ID"].values

False

In [160]:
20 in tg_list

False

In [192]:
train = pd.DataFrame(bin_comp_split.test_set)

In [193]:
label_data

,TF ID,Target ID
0,5,456
1,5,3527
2,5,4626
3,5,7756
4,5,12597
...,...,...
1417,36454,35760
1418,36459,7756
1420,36747,15381
1421,36966,8342


In [198]:
train_pos = train.loc[train.iloc[:,2] == 1]

for i, row in train_pos.iterrows():
    tf, tg, _ = row
    combi = (tf, tg)
    if label_data[(label_data["TF ID"] == tf )& (label_data["Target ID"] == tg)].shape[0] < 1:
        print("false")

In [188]:
exp = "EXP/expression.tsv"
os.path.exists(exp)
exp = pd.read_table(exp, index_col=0)

Index(['DRX019488', 'DRX026439', 'DRX039919', 'DRX039923', 'DRX039924',
       'DRX039925', 'DRX066823', 'DRX066824', 'DRX066825', 'DRX078166',
       ...
       'SRX7786231', 'SRX7868575', 'SRX7868576', 'SRX7868577', 'SRX7869407',
       'SRX7869408', 'SRX7869409', 'SRX7869420', 'SRX7869421', 'SRX7869422'],
      dtype='object', length=1343)

## splitting the TF and TGs 